In [ ]:
!pip install pyspark


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import functions  as F
from pyspark.sql.window import Window

In [6]:
spark = SparkSession.builder.appName("covid_analysis").getOrCreate()

In [ ]:
df = spark.read.csv("covid-data.csv", header = True)


In [ ]:
result_df = (df.filter(F.col("date").between("2021-03-25", "2021-03-31"))
               .filter(F.col("location") == "Russia")
               .withColumn("Число", F.date_format(F.col("date"), "dd"))
               .withColumn("Кол-во новых случаев вчера", F.col("new_cases")))

window_spec = Window.orderBy(F.col("Число"))
result_df = result_df.withColumn("Кол-во новых случаев сегодня", F.lag(F.col("new_cases")).over(window_spec))
result_df.show()

In [24]:
result = (result_df
                   .withColumn("Дельта", F.col("Кол-во новых случаев вчера") - F.col("Кол-во новых случаев сегодня"))
                   .select("Число", "Кол-во новых случаев вчера", "Кол-во новых случаев сегодня", "Дельта")
                   .orderBy(F.col("Число").desc()))
result.show()

+-----+--------------------------+----------------------------+------+
|Число|Кол-во новых случаев вчера|Кол-во новых случаев сегодня|Дельта|
+-----+--------------------------+----------------------------+------+
|   31|                    8156.0|                      8162.0|  -6.0|
|   30|                    8162.0|                      8589.0|-427.0|
|   29|                    8589.0|                      8979.0|-390.0|
|   28|                    8979.0|                      8783.0| 196.0|
|   27|                    8783.0|                      9073.0|-290.0|
|   26|                    9073.0|                      9128.0| -55.0|
|   25|                    9128.0|                        NULL|  NULL|
+-----+--------------------------+----------------------------+------+

